In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:
path_df = "News_dataset.pickle"

with open(path_df, 'rb') as data:
    data = pickle.load(data)

In [3]:
data.head()

,File_Name,Content,Category,Complete_Filename,id,News_length
0,001.txt,Ad sales boost Time Warner profit\r\n\r\nQuart...,business,001.txt-business,1,2569
1,002.txt,Dollar gains on Greenspan speech\r\n\r\nThe do...,business,002.txt-business,1,2257
2,003.txt,Yukos unit buyer faces loan claim\r\n\r\nThe o...,business,003.txt-business,1,1557
3,004.txt,High fuel prices hit BA's profits\r\n\r\nBriti...,business,004.txt-business,1,2421
4,005.txt,Pernod takeover talk lifts Domecq\r\n\r\nShare...,business,005.txt-business,1,1575


In [4]:

data.loc[1]['Content']

'Dollar gains on Greenspan speech\r\n\r\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\r\n\r\nAnd Alan Greenspan highlighted the US government\'s willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan\'s speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman\'s taking a much more sanguine view on the current account deficit than he\'s taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He\'s taking a longer-term view, laying out a set of cond

## 1.  Text cleaning and preparation

In [5]:
data['Clean_Content']=data['Content'].apply(lambda x: x.replace("\r"," "))
data['Clean_Content']=data['Clean_Content'].apply(lambda x: x.replace("\n"," "))
data['Clean_Content']=data['Clean_Content'].apply(lambda x: x.replace("   "," "))
data['Clean_Content']=data['Clean_Content'].apply(lambda x: x.replace('"',''))
data['Clean_Content']

0       Ad sales boost Time Warner profit  Quarterly p...
1       Dollar gains on Greenspan speech  The dollar h...
2       Yukos unit buyer faces loan claim  The owners ...
3       High fuel prices hit BA's profits  British Air...
4       Pernod takeover talk lifts Domecq  Shares in U...
                              ...                        
2220    BT program to beat dialler scams  BT is introd...
2221    Spam e-mails tempt net shoppers  Computer user...
2222    Be careful how you code  A new European direct...
2223    US cyber security chief resigns  The man makin...
2224    Losing yourself in online gaming  Online role ...
Name: Clean_Content, Length: 2225, dtype: object

In [6]:
data.loc[1]['Clean_Content']

"Dollar gains on Greenspan speech  The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.  And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan's speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time, said Robert Sinche, head of currency strategy at Bank of America in New York. He's taking a longer-term view, laying out a set of conditions under which t

In [7]:
data.head()

,File_Name,Content,Category,Complete_Filename,id,News_length,Clean_Content
0,001.txt,Ad sales boost Time Warner profit\r\n\r\nQuart...,business,001.txt-business,1,2569,Ad sales boost Time Warner profit Quarterly p...
1,002.txt,Dollar gains on Greenspan speech\r\n\r\nThe do...,business,002.txt-business,1,2257,Dollar gains on Greenspan speech The dollar h...
2,003.txt,Yukos unit buyer faces loan claim\r\n\r\nThe o...,business,003.txt-business,1,1557,Yukos unit buyer faces loan claim The owners ...
3,004.txt,High fuel prices hit BA's profits\r\n\r\nBriti...,business,004.txt-business,1,2421,High fuel prices hit BA's profits British Air...
4,005.txt,Pernod takeover talk lifts Domecq\r\n\r\nShare...,business,005.txt-business,1,1575,Pernod takeover talk lifts Domecq Shares in U...


In [8]:
data['Clean_Content']=data['Clean_Content'].apply(lambda x: x.lower())
data['Clean_Content']

0       ad sales boost time warner profit  quarterly p...
1       dollar gains on greenspan speech  the dollar h...
2       yukos unit buyer faces loan claim  the owners ...
3       high fuel prices hit ba's profits  british air...
4       pernod takeover talk lifts domecq  shares in u...
                              ...                        
2220    bt program to beat dialler scams  bt is introd...
2221    spam e-mails tempt net shoppers  computer user...
2222    be careful how you code  a new european direct...
2223    us cyber security chief resigns  the man makin...
2224    losing yourself in online gaming  online role ...
Name: Clean_Content, Length: 2225, dtype: object

In [9]:
import string 
import nltk
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tejas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [11]:
exclude_punct=set(string.punctuation)
stop_wards=set(stopwords.words('English'))

def cleaning_text(text):
    tokens=word_tokenize(text)
    filtered_text=[word for word in tokens if word.lower() not in stop_wards and word not in exclude_punct]
    return ' '.join(filtered_text)

data['Clean_Content']=data['Clean_Content'].apply(cleaning_text)
data

,File_Name,Content,Category,Complete_Filename,id,News_length,Clean_Content
0,001.txt,Ad sales boost Time Warner profit\r\n\r\nQuart...,business,001.txt-business,1,2569,ad sales boost time warner profit quarterly pr...
1,002.txt,Dollar gains on Greenspan speech\r\n\r\nThe do...,business,002.txt-business,1,2257,dollar gains greenspan speech dollar hit highe...
2,003.txt,Yukos unit buyer faces loan claim\r\n\r\nThe o...,business,003.txt-business,1,1557,yukos unit buyer faces loan claim owners embat...
3,004.txt,High fuel prices hit BA's profits\r\n\r\nBriti...,business,004.txt-business,1,2421,high fuel prices hit ba 's profits british air...
4,005.txt,Pernod takeover talk lifts Domecq\r\n\r\nShare...,business,005.txt-business,1,1575,pernod takeover talk lifts domecq shares uk dr...
...,...,...,...,...,...,...,...
2220,397.txt,BT program to beat dialler scams\r\n\r\nBT is ...,tech,397.txt-tech,1,2526,bt program beat dialler scams bt introducing t...
2221,398.txt,Spam e-mails tempt net shoppers\r\n\r\nCompute...,tech,398.txt-tech,1,2294,spam e-mails tempt net shoppers computer users...
2222,399.txt,Be careful how you code\r\n\r\nA new European ...,tech,399.txt-tech,1,6297,careful code new european directive could put ...
2223,400.txt,US cyber security chief resigns\r\n\r\nThe man...,tech,400.txt-tech,1,2323,us cyber security chief resigns man making sur...


In [12]:
data['Content'].loc[1]

'Dollar gains on Greenspan speech\r\n\r\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\r\n\r\nAnd Alan Greenspan highlighted the US government\'s willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan\'s speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman\'s taking a much more sanguine view on the current account deficit than he\'s taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He\'s taking a longer-term view, laying out a set of cond

In [13]:
data['Clean_Content'].loc[1]

"dollar gains greenspan speech dollar hit highest level euro almost three months federal reserve head said us trade deficit set stabilise alan greenspan highlighted us government 's willingness curb spending rising household savings factors may help reduce late trading new york dollar reached 1.2871 euro 1.2974 thursday market concerns deficit hit greenback recent months friday federal reserve chairman mr greenspan 's speech london ahead meeting g7 finance ministers sent dollar higher earlier tumbled back worse-than-expected us jobs data think chairman 's taking much sanguine view current account deficit 's taken time said robert sinche head currency strategy bank america new york 's taking longer-term view laying set conditions current account deficit improve year next worries deficit concerns china however remain china 's currency remains pegged dollar us currency 's sharp falls recent months therefore made chinese export prices highly competitive calls shift beijing 's policy fallen

In [14]:
data['Clean_Content']=data['Clean_Content'].apply(lambda x: x.replace("'s",""))
data['Clean_Content'].loc[1]

'dollar gains greenspan speech dollar hit highest level euro almost three months federal reserve head said us trade deficit set stabilise alan greenspan highlighted us government  willingness curb spending rising household savings factors may help reduce late trading new york dollar reached 1.2871 euro 1.2974 thursday market concerns deficit hit greenback recent months friday federal reserve chairman mr greenspan  speech london ahead meeting g7 finance ministers sent dollar higher earlier tumbled back worse-than-expected us jobs data think chairman  taking much sanguine view current account deficit  taken time said robert sinche head currency strategy bank america new york  taking longer-term view laying set conditions current account deficit improve year next worries deficit concerns china however remain china  currency remains pegged dollar us currency  sharp falls recent months therefore made chinese export prices highly competitive calls shift beijing  policy fallen deaf ears despi

### Stemming and Lemmatization

In [15]:
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')

nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tejas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tejas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tejas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

#### 1st method lemmatization

In [16]:
from nltk.stem import WordNetLemmatizer

In [17]:
def lemmatize_text(text):
    tokens=word_tokenize(text)
    lemma=WordNetLemmatizer()
    lemma_text=[lemma.lemmatize(word) for word in tokens]
    return ' '.join(lemma_text)

data['Clean_Content']=data['Clean_Content'].apply(lemmatize_text)
data

,File_Name,Content,Category,Complete_Filename,id,News_length,Clean_Content
0,001.txt,Ad sales boost Time Warner profit\r\n\r\nQuart...,business,001.txt-business,1,2569,ad sale boost time warner profit quarterly pro...
1,002.txt,Dollar gains on Greenspan speech\r\n\r\nThe do...,business,002.txt-business,1,2257,dollar gain greenspan speech dollar hit highes...
2,003.txt,Yukos unit buyer faces loan claim\r\n\r\nThe o...,business,003.txt-business,1,1557,yukos unit buyer face loan claim owner embattl...
3,004.txt,High fuel prices hit BA's profits\r\n\r\nBriti...,business,004.txt-business,1,2421,high fuel price hit ba profit british airway b...
4,005.txt,Pernod takeover talk lifts Domecq\r\n\r\nShare...,business,005.txt-business,1,1575,pernod takeover talk lift domecq share uk drin...
...,...,...,...,...,...,...,...
2220,397.txt,BT program to beat dialler scams\r\n\r\nBT is ...,tech,397.txt-tech,1,2526,bt program beat dialler scam bt introducing tw...
2221,398.txt,Spam e-mails tempt net shoppers\r\n\r\nCompute...,tech,398.txt-tech,1,2294,spam e-mail tempt net shopper computer user ac...
2222,399.txt,Be careful how you code\r\n\r\nA new European ...,tech,399.txt-tech,1,6297,careful code new european directive could put ...
2223,400.txt,US cyber security chief resigns\r\n\r\nThe man...,tech,400.txt-tech,1,2323,u cyber security chief resigns man making sure...


In [18]:
data[['Content','Clean_Content']]

,Content,Clean_Content
0,Ad sales boost Time Warner profit\r\n\r\nQuart...,ad sale boost time warner profit quarterly pro...
1,Dollar gains on Greenspan speech\r\n\r\nThe do...,dollar gain greenspan speech dollar hit highes...
2,Yukos unit buyer faces loan claim\r\n\r\nThe o...,yukos unit buyer face loan claim owner embattl...
3,High fuel prices hit BA's profits\r\n\r\nBriti...,high fuel price hit ba profit british airway b...
4,Pernod takeover talk lifts Domecq\r\n\r\nShare...,pernod takeover talk lift domecq share uk drin...
...,...,...
2220,BT program to beat dialler scams\r\n\r\nBT is ...,bt program beat dialler scam bt introducing tw...
2221,Spam e-mails tempt net shoppers\r\n\r\nCompute...,spam e-mail tempt net shopper computer user ac...
2222,Be careful how you code\r\n\r\nA new European ...,careful code new european directive could put ...
2223,US cyber security chief resigns\r\n\r\nThe man...,u cyber security chief resigns man making sure...


In [19]:
print(data['Content'].loc[1])
print("=====================================================After Cleaning===================================================")
print(data['Clean_Content'].loc[1])

Dollar gains on Greenspan speech

The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.

And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan's speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He's taking a longer-term view, laying out a set of conditions under which

In [22]:

wordnet_lemmatizer = WordNetLemmatizer()
nrows = len(data)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

df['Content_Parsed_5'] = lemmatized_text_list    

In [23]:
df['Content_Parsed_5']


0       ad sales boost time warner profit quarterly pr...
1       dollar gain on greenspan speech the dollar hav...
2       yukos unit buyer face loan claim the owners of...
3       high fuel price hit ba profit british airways ...
4       pernod takeover talk lift domecq share in uk d...
                              ...                        
2220    bt program to beat dialler scam bt be introduc...
2221    spam e-mail tempt net shoppers computer users ...
2222    be careful how you code a new european directi...
2223    us cyber security chief resign the man make su...
2224    lose yourself in online game online role play ...
Name: Content_Parsed_5, Length: 2225, dtype: object

#### 2nd method for lemmatization

In [24]:
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    lemmatized_text = lemmatize_sentence(df.loc[row]['Content_Parsed_4'])
    lemmatized_text_list.append(lemmatized_text)

df['Content_Parsed_5'] = lemmatized_text_list    

In [25]:
df['Content_Parsed_5']

0       ad sale boost time warner profit quarterly pro...
1       dollar gain on greenspan speech the dollar hav...
2       yukos unit buyer face loan claim the owner of ...
3       high fuel price hit ba profit british airway h...
4       pernod takeover talk lift domecq share in uk d...
                              ...                        
2220    bt program to beat dialler scam bt be introduc...
2221    spam e-mails tempt net shopper computer user a...
2222    be careful how you code a new european directi...
2223    us cyber security chief resign the man make su...
2224    lose yourself in online gaming online role pla...
Name: Content_Parsed_5, Length: 2225, dtype: object

### b) Use any 1 method for stop word

In [26]:
#Downloading

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/dipali/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
#Removing stop words

stop_words = list(stopwords.words('english'))

#### 1st Method

In [28]:

df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

/tmp/ipykernel_4042/2995545048.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


In [29]:
df.loc[5]['Content_Parsed_6']

'japan narrowly escape recession japan economy teeter   brink   technical recession   three month  september figure show revised figure indicate growth   01 % -   similar-sized contraction   previous quarter   annual basis  data suggest annual growth   02 % suggest  much  hesitant recovery   previously  think  common technical definition   recession  two successive quarter  negative growth  government  keen  play   worrying implication   data  maintain  view  japan economy remain   minor adjustment phase   upward climb    monitor development carefully say economy minister heizo takenaka    face   strengthen yen make export less competitive  indication  weaken economic condition ahead observer  less sanguine  paint  picture   recovery much patchy  previously think say paul sheard economist  lehman brother  tokyo improvement   job market apparently  yet  fee   domestic demand  private consumption   02 %   third quarter'

#### 2nd Method

In [30]:
stop_list_final=[]
nrows = len(df)
stopwords_english = stopwords.words('english') 

for row in range(0, nrows):
    
    # Create an empty list containing no stop words
    stop_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_5']
    text_words = text.split(" ")

    # Iterate through every word to remove stopwords
    for word in text_words:
        if (word not in stopwords_english): 
          stop_list.append(word)
        
    # Join the list
    stop_text = " ".join(stop_list)
    
    # Append to the list containing the texts
    stop_list_final.append(stop_text)

df['Content_Parsed_6'] = stop_list_final   
       

In [31]:
df.loc[5]['Content_Parsed_6']

'japan narrowly escape recession japan economy teeter brink technical recession three month september figure show revised figure indicate growth 01 % - similar-sized contraction previous quarter annual basis data suggest annual growth 02 % suggest much hesitant recovery previously think common technical definition recession two successive quarter negative growth government keen play worrying implication data maintain view japan economy remain minor adjustment phase upward climb monitor development carefully say economy minister heizo takenaka face strengthen yen make export less competitive indication weaken economic condition ahead observer less sanguine paint picture recovery much patchy previously think say paul sheard economist lehman brother tokyo improvement job market apparently yet fee domestic demand private consumption 02 % third quarter'

In [32]:
#Checking data

df.head(1)

,File_Name,Content,Category,Complete_Filename,id,News_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,001.txt,Ad sales boost Time Warner profit\r\n\r\nQuart...,business,001.txt-business,1,2569,Ad sales boost Time Warner profit Quarterly pr...,ad sales boost time warner profit quarterly pr...,ad sales boost time warner profit quarterly pr...,ad sales boost time warner profit quarterly pr...,ad sale boost time warner profit quarterly pro...,ad sale boost time warner profit quarterly pro...


In [33]:
#Removing the old content_parsed columns

list_columns = ["File_Name", "Category", "Complete_Filename", "Content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [34]:
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed
0,001.txt,business,001.txt-business,Ad sales boost Time Warner profit\r\n\r\nQuart...,ad sale boost time warner profit quarterly pro...
1,002.txt,business,002.txt-business,Dollar gains on Greenspan speech\r\n\r\nThe do...,dollar gain greenspan speech dollar hit high l...
2,003.txt,business,003.txt-business,Yukos unit buyer faces loan claim\r\n\r\nThe o...,yukos unit buyer face loan claim owner embattl...
3,004.txt,business,004.txt-business,High fuel prices hit BA's profits\r\n\r\nBriti...,high fuel price hit ba profit british airway b...
4,005.txt,business,005.txt-business,Pernod takeover talk lifts Domecq\r\n\r\nShare...,pernod takeover talk lift domecq share uk drin...


## 2. Label coding

In [35]:
#Generating new column for Category codes

category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

In [36]:
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed,Category_Code
0,001.txt,business,001.txt-business,Ad sales boost Time Warner profit\r\n\r\nQuart...,ad sale boost time warner profit quarterly pro...,0
1,002.txt,business,002.txt-business,Dollar gains on Greenspan speech\r\n\r\nThe do...,dollar gain greenspan speech dollar hit high l...,0
2,003.txt,business,003.txt-business,Yukos unit buyer faces loan claim\r\n\r\nThe o...,yukos unit buyer face loan claim owner embattl...,0
3,004.txt,business,004.txt-business,High fuel prices hit BA's profits\r\n\r\nBriti...,high fuel price hit ba profit british airway b...,0
4,005.txt,business,005.txt-business,Pernod takeover talk lifts Domecq\r\n\r\nShare...,pernod takeover talk lift domecq share uk drin...,0


## 3. Train - test split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

## 4. Text representation

TF-IDF Vectors

unigrams & bigrams corresponding to a particular category


In [38]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [39]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1891, 300)
(334, 300)


In [41]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names_out())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'business' category:
  . Most correlated unigrams:
. price
. market
. economy
. growth
. bank
  . Most correlated bigrams:
. last year
. year old

# 'entertainment' category:
  . Most correlated unigrams:
. best
. music
. star
. award
. film
  . Most correlated bigrams:
. mr blair
. prime minister

# 'politics' category:
  . Most correlated unigrams:
. blair
. party
. election
. tory
. labour
  . Most correlated bigrams:
. prime minister
. mr blair

# 'sport' category:
  . Most correlated unigrams:
. side
. player
. team
. game
. match
  . Most correlated bigrams:
. say mr
. year old

# 'tech' category:
  . Most correlated unigrams:
. mobile
. software
. technology
. computer
. user
  . Most correlated bigrams:
. year old
. say mr



In [42]:
bigrams

['tell bbc', 'last year', 'mr blair', 'prime minister', 'year old', 'say mr']

Unigrams are more relevnat to the category as compared with bigrams

In [2]:
# X_train
with open('X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('/content/drive/My Drive/Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('/content/drive/My Drive/Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)

NameError: name 'pickle' is not defined